# 1. Data Acquistion

In [1]:
import urllib.request
import os

#Creating data folder
if not os.path.exists('./data'):
    os.makedirs('./data')
    
# Downloading dataset using the url that hosts it
kaggle_url = 'https://github.com/sundeepblue/movie_rating_prediction/raw/master/movie_metadata.csv'
if not os.path.exists('/data/kaggle_dataset.csv'):
    data = urllib.request.urlretrieve(kaggle_url, './data/kaggle_dataset.csv')

 

In [2]:
import gzip

#Obtaining IMDB's text files that are zipped winrar
imdb_url_prefix = 'https://ftp.funet.fi/pub/mirrors/ftp.imdb.com/pub/frozendata/'
imdb_files_list = ['genres.list.gz', 'ratings.list.gz']

for name in imdb_files_list:
    if not os.path.exists('./data/' + name):
        data = urllib.request.urlretrieve(imdb_url_prefix + name, './data/' + name)
        urllib.request.urlcleanup() # urllib fails to download two files from a ftp source. This fixes the bug!
        with gzip.open('./data/' + name, 'rb') as comp_file, open('./data/' + name[:-3], 'wb') as reg_file:   #'w' alone doesn't work because you are reading binary, so you have to write binary, so use 'wb'
            file_content = comp_file.read()
            reg_file.write(file_content)
            
# Error WHY DO I HAVE TO RUN THIS TWICE< IT WONT DOWNLOAD BOTH FILES SAME TIME

In [3]:
imdb_url = 'https://anaconda.org/BigGorilla/datasets/1/download/imdb_dataset.csv'

if not os.path.exists('./data/imdb_dataset.csv'):  #Avoids downloading if it already exists
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'}
    req = urllib.request.Request(url=imdb_url, headers=headers) #To fix HTTP error 403: Forbidden
    data = urllib.request.urlopen(req).read()
    
    with open("./data/imdb_dataset.csv", "wb") as f:
        f.write(data)

# 2.Data Extraction

The “Kaggle 5000 Movie Dataset” is stored in a .csv file which is already structured and ready to use. On the other hand, the “IMDB Plain Text Data” is a collection of semi-structured text files that need to be processed to extract the data. A quick look at the first few lines of each files shows that each file has a different format and has to be handled separately.

**Contents of "ratings.list" file**

In [4]:
with open("./data/ratings.list") as f:
    head = [next(f) for x in range(38)]
print(''.join(head[28:38]))  #Skip the first 28 lines cuz they are descriptive headers

      0000000125  1888533   9.2  The Shawshank Redemption (1994)
      0000000125  1289428   9.2  The Godfather (1972)
      0000000124  889607   9.0  The Godfather: Part II (1974)
      0000000124  1864164   9.0  The Dark Knight (2008)
      0000000133  518449   8.9  12 Angry Men (1957)
      0000000133  971107   8.9  Schindler's List (1993)
      0000000123  1477112   8.9  Pulp Fiction (1994)
      0000000124  1349449   8.9  The Lord of the Rings: The Return of the King (2003)
      0000000123  559468   8.8  Il buono, il brutto, il cattivo (1966)
      0000000133  1513600   8.8  Fight Club (1999)



**Conents of "genres.list" file**

In [5]:
with open('./data/genres.list') as f:
    head = [next(f) for x in range(392)]
print(''.join(head[382:392]))  # skipping the first 382 lines as they are descriptive header


"!Next?" (1994)						Documentary
"#1 Single" (2006)					Reality-TV
"#15SecondScare" (2015)					Horror
"#15SecondScare" (2015)					Short
"#15SecondScare" (2015)					Thriller
"#15SecondScare" (2015) {Who Wants to Play with the Rabbit? (#1.2)}	Drama
"#15SecondScare" (2015) {Who Wants to Play with the Rabbit? (#1.2)}	Horror
"#15SecondScare" (2015) {Who Wants to Play with the Rabbit? (#1.2)}	Short



**Extracing info from "genres.list"**

In [10]:
import re
import pandas as pd

with open("./data/genres.list") as genres_file:
    raw_content = genres_file.readlines()
    genres_list = []         #List that will contain lists for each [movie,year,genre]
    content = raw_content[384:]
    for line in content:
            #print(line.strip())
            m = re.match(r'"?(.*[^"])"? \(((?:\d|\?){4})(?:/\w*)?\).*\s((?:\w|-)+)', line.strip())
            #print(m.group(1), m.group(2), m.group(3))
            genres_list.append([m.group(1), m.group(2), m.group(3)])
            
    genres_data = pd.DataFrame(genres_list, columns=['movie', 'year', 'genre'])

In [12]:
genres_data

,movie,year,genre
0,!Next?,1994,Documentary
1,#1 Single,2006,Reality-TV
2,#15SecondScare,2015,Horror
3,#15SecondScare,2015,Short
4,#15SecondScare,2015,Thriller
...,...,...,...
2658936,überRICH,2017,Comedy
2658937,überRICH,2017,Short
2658938,üç,2012,Adventure
2658939,üç,2012,Comedy


**Extracing the info from "ratings.list"**

In [27]:
with open("./data/ratings.list") as f:
    raw_content = f.readlines()
    ratings_list = []
    content = raw_content[28:]
    #print(content)
    
    for line in content:
        m = re.match(r'(?:\d|\.|\*){10}\s+\d+\s+(1?\d\.\d)\s"?(.*[^"])"? \(((?:\d|\?){4})(?:/\w*)?\)', line.strip())
        if m is None: continue
        ratings_list.append([m.group(2), m.group(3), m.group(1)])
        
    ratings_data = pd.DataFrame(ratings_list, columns=["movie", 'year', 'rating'])

Note that one has to repeat the information extraction procedure for other data files as well if he is interested in their content. For now (and to keep the tutorial simple), we assume that we are only interested in genres and ratings of movies. The above code snippets store the extracted data on these two attributes into two dataframes (namely, genres_list and ratings_list).

# 3.Data Profiling & Cleaning

The high-level goal in this stage of data prepration is to look into the data that we have acquired and extracted so far. This helps us to get familiar with data, understand in what ways the data needs cleaning or transformation, and finally enables us to prepare the data for the following steps of the data integration task.

In [18]:
import pandas as pd

#Load the Kaggle dataset from the .csv file
kaggle_data = pd.read_csv("./data/kaggle_dataset.csv")

In [28]:
#Let’s start by finding out how many movies are listed in each dataframe.

print ('Number of movies in kaggle_data: {}'.format(kaggle_data.shape[0]))
print ('Number of movies in genres_data: {}'.format(genres_data.shape[0]))
print ('Number of movies in ratings_data: {}'.format(ratings_data.shape[0]))

Number of movies in kaggle_data: 5043
Number of movies in genres_data: 2658941
Number of movies in ratings_data: 789415


In [26]:
#We can also check to see if we have duplicates (i.e., a movie appearing more than once) in the data. 
#We consider an entry duplicate if we can find another entry with the same movie title and production year

print ('Number of duplicates in kaggle_data: {}'.format(
    sum(kaggle_data.duplicated(subset=['movie_title', 'title_year'], keep=False))))

print ('Number of duplicates in genres_data: {}'.format(
    sum(genres_data.duplicated(subset=['movie', 'year'], keep=False))))
       
print ('Number of duplicates in ratings_data: {}'.format(
    sum(ratings_data.duplicated(subset=['movie', 'year'], keep=False))))

Number of duplicates in kaggle_data: 241
Number of duplicates in genres_data: 2031322
Number of duplicates in ratings_data: 342815


**Dealing with Duplicates (cleaning)**

One way of dealing with duplicates we are going to use a simple method for dealing with duplicates 
and that is to only keep the first occurrence of a duplicated entry and remove the rest.

In [33]:
#copy so we don't affect the original data
kaggle_data = kaggle_data.drop_duplicates(subset=['movie_title', 'title_year'], keep='first').copy()
genres_data = genres_data.drop_duplicates(subset=['movie', 'year'], keep='first').copy()
ratings_data = ratings_data.drop_duplicates(subset=['movie', 'year'], keep='first').copy()

print ('Number of duplicates in kaggle_data: {}'.format(
    sum(kaggle_data.duplicated(subset=['movie_title', 'title_year'], keep=False))))

print ('Number of duplicates in genres_data: {}'.format(
    sum(genres_data.duplicated(subset=['movie', 'year'], keep=False))))
       
print ('Number of duplicates in ratings_data: {}'.format(
    sum(ratings_data.duplicated(subset=['movie', 'year'], keep=False))))

Number of duplicates in kaggle_data: 0
Number of duplicates in genres_data: 0
Number of duplicates in ratings_data: 0


**Normalizing the text (cleaning)**

The key attribute that we will use to integrate our movie datasets is the movie titles. So it is important to normalize these titles. The following code snippet makes all movie titles lower case, and then removes certain characters such as “‘” and “?”, and replaces some other special characters (e.g., “&” is replaced with “and”).

In [34]:
def preprocess_title(title):
    title = title.lower()
    title = title.replace("?", '')
    title = title.replace(',', '')
    title = title.replace("'", '') 
    title = title.replace('&', 'and')
    title = title.replace('utf-8', 'ignore')
    return title.strip()

kaggle_data['norm_movie_title'] = kaggle_data['movie_title'].map(preprocess_title)
genres_data['norm_movie'] = genres_data['movie'].map(preprocess_title)
ratings_data['norm_movie'] = ratings_data['movie'].map(preprocess_title)

#Created a new column with the normalized movie names 

In [41]:
print(kaggle_data['movie_title'].sample(3,random_state=0))
kaggle_data['norm_movie_title'].sample(3, random_state=0)

4422       Unsullied 
1022       Fair Game 
3631    The Wackness 
Name: movie_title, dtype: object


4422       unsullied
1022       fair game
3631    the wackness
Name: norm_movie_title, dtype: object

In [42]:
kaggle_data.sample(3, random_state=0)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,norm_movie_title
4422,Color,Simeon Rice,6.0,93.0,6.0,56.0,Lisa Brave,393.0,NaN,Action|Horror|Thriller,...,English,USA,R,1500000.0,2014.0,191.0,5.5,2.35,307,unsullied
1022,Color,Doug Liman,214.0,108.0,218.0,405.0,Ty Burrell,6000.0,9528092.0,Biography|Drama|Thriller,...,English,USA,PG-13,22000000.0,2010.0,3000.0,6.8,2.35,9000,fair game
3631,Color,Jonathan Levine,147.0,99.0,129.0,362.0,Aaron Yoo,976.0,2077046.0,Comedy|Drama|Romance,...,English,USA,R,6000000.0,2008.0,617.0,7.0,2.35,0,the wackness


Looking at the data guides us to decide in what ways we might want to clean the data. For instance, the small sample data shown above, reveals that the title_year attribute is stored as floats (i.e., rational numbers). We can add another cleaning step to transform the title_year into strings and replace the missing title years with symbol “?”.

In [43]:
def preprocess_year(year):
    if pd.isnull(year):
        return '?'
    else:
        return str(int(year))
    
kaggle_data['norm_title_year'] = kaggle_data['title_year'].map(preprocess_year)
kaggle_data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,norm_movie_title,norm_title_year
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000,avatar,2009
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0,pirates of the caribbean: at worlds end,2007
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000,spectre,2015
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000,the dark knight rises,2012
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0,star wars: episode vii - the force awakens,?


# 4.Data Matching & Merging